In [8]:
import sys
import os

# Get the root directory of the project
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the root directory to the system path
sys.path.insert(0, project_root)

In [6]:
import pandas as pd 
import os
import json

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# import seaborn as sns
# sns.set_theme()

In [9]:
from explainable_dataset import *
from viz_utils import create_highlighted_passage
from IPython.display import display, HTML

def highlight_passage(tokens, binary_list):    
    highlighted_passage = create_highlighted_passage(tokens,
                                                     binary_list,
                                                     None,
                                                     'green',
                                                     'linear')
    display(HTML("\n".join(highlighted_passage)))
    
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')


file_path = '../data/29_random_samples_explained.jsonl'
dataset_me = ExplanationsDataset(file_path, tokenizer, decode_positive_as_list=True)

file_path = '../data/29_random_samples_Meta-Llama-3.1-8B-Instruct.jsonl'
dataset_llama = ExplanationsDataset(file_path, tokenizer, decode_positive_as_list=True)

# Iterate through the data
for i in range(len(dataset_llama)):
    print(dataset_me[i]["query"])
    
    print("Me")
    d = dataset_me[i]
    highlight_passage(d['tokenized_positive_decoded'], d['rationales'])
    
    print("Llma")  
    d = dataset_llama[i]
    highlight_passage(d['tokenized_positive_decoded'], d['rationales'])
    
    print("\n")

what is a system hardware
Me


Llma




meaning of the name lincoln
Me


Llma


AssertionError: Selected span 'The name of an early Roman settlement in England' was not found in the text: American Meaning: The name Lincoln is an American baby name. In American the meaning of the name Lincoln is: From the settlement by the pool. English Meaning: The name Lincoln is an English baby name.In English the meaning of the name Lincoln is: Roman colony at the pool.Lakeside colony. The name of an early Roman settlement in England, used as both a surname and a first name. Famous Bearer: American president Abraham Lincoln.he name Lincoln is an English baby name. In English the meaning of the name Lincoln is: Roman colony at the pool. Lakeside colony. The name of an early Roman settlement in England, used as both a surname and a first name.

In [ ]:

dataset_paths = [
    '29_random_samples_explained.jsonl',
    '29_random_samples_Meta-Llama-3.1-8B-Instruct.jsonl',
    '29_random_samples_gpt-4o-2024-08-06.jsonl',
    '29_random_samples_gpt-4o-mini-2024-07-18.jsonl',
]

datasets = {}
for dataset_path in dataset_paths:
    dataset_path = "../data/" + dataset_path
    d = ExplanationsDataset(dataset_path, tokenizer, decode_positive_as_list=True)
    dataset_name = dataset_path.removeprefix("../data/29_random_samples_").removesuffix(".jsonl")
    datasets[dataset_name] = d

datasets

In [ ]:
import pandas as pd

# Initialize the DataFrame to hold the data
columns = ['query']  # Start with 'query'
# Add columns dynamically based on the dataset keys
columns.extend([f"tokenized_positive_decoded_{key}" for key in datasets.keys()])
columns.extend([f"rationales_{key}" for key in datasets.keys()])

df = pd.DataFrame(columns=columns)

# Assuming all datasets have the same length
for i in range(len(next(iter(datasets.values())))):  # Use length of the first dataset
    row_data = {'query': datasets[next(iter(datasets))][i]['query']}  # Initialize with query
    
    # Iterate through each dataset in the dictionary
    for key, dataset in datasets.items():
        row_data[f"tokenized_positive_decoded_{key}"] = dataset[i]['tokenized_positive_decoded']
        row_data[f"rationales_{key}"] = dataset[i]['rationales']
    
    # Append the data to the DataFrame
    df = pd.concat([df, pd.DataFrame([row_data])], ignore_index=True)

# Output the DataFrame
df


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd


def compute_stats(rationales_label):
    # Initialize variables to store the cumulative metrics
    precisions = []
    recalls = []
    f1_scores = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Get the rationales from both datasets
        rationales_llama = row["rationales_" + rationales_label]
        rationales_me = row['rationales_explained']

        y_true = rationales_me.tolist()
        y_pred = rationales_llama.tolist() 

        # Calculate precision, recall, and F1 score for this pair of rationales
        try:
            precision = precision_score(y_true, y_pred)
            recall = recall_score(y_true, y_pred)
            f1 = f1_score(y_true, y_pred)
        except ValueError as e:
            print(f"{index}: {e}")
            print()
            continue
        # Append the scores to the lists
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Calculate the average of each metric
    avg_precision = sum(precisions) / len(precisions)
    avg_recall = sum(recalls) / len(recalls)
    avg_f1 = sum(f1_scores) / len(f1_scores)

    # Print the averages
    print(f"\n\tStats for {rationales_label}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1 Score: {avg_f1:.4f}")

    
for model_name in datasets.keys():
    compute_stats(model_name)
    

In [ ]:
print(df.iloc[24]["tokenized_positive_decoded_llama"])
print(df.iloc[23]["tokenized_positive_decoded_me"])